# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Grande Instruct** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Interact with the model](#B.-Interact-with-the-model)
   3. [Give it an instruction](#C.-Give-it-an-instruction)
   4. [Create your own product description!](#D.-Create-your-own-product-description!)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-grande-instruct-v1-1-033-92fee9d4f82f3b02a76ae298452f7378"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.114'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for ai21: filename=ai21-1.0.5-py3-none-any.whl size=16622 sha256=4fec9b8bd97e1155e91884a8c87ebdc9b9005d1f4629dd1237c2bf80424dd96c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ad/0d/cc/60f67cd2b7c536fb97df70a5bfc454080d76c195a033e036d4
Successfully built ai21


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
endpoint_name = "j2-grande-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

----------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Grande Instruct as a smart auto-completion algorithm: give it some text as input and it will generate relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [9]:
response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Give it an instruction

This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

For this notebook, we will apply the model to creating a product description for an eCommerce site. Simply tell the model what you want it to do with a simple instruction.

In [12]:
instruction = """
Please summary the text

Text: 美國聯儲局周三（5月3日）一如預期加息0.25厘，議息聲明暗示或停止進一步加息。聯儲局主席鮑威爾（Jerome Powell ）在會後於記者會上稱，他不排除美國出現經濟衰退的可能性，但相信能夠避免衰退。他又指出，局方在壓低通脹方面仍有漫長的路要走，如果通脹仍然偏高，當局將不會減息，而聯邦公開市場委員會（FOMC）的通脹前景展望，並不支持減息。

鮑威爾表示，委員會曾討論暫停加息，今次會議並無作出這個決定，但認為正接近甚至可能已達到終點，可能會暫停一段時間，未來將會逐次會議評估。聯儲局正在設法實現有效的限制性利率水平，局方必須在加息幅度不夠大，以及加息過度的風險之間取得平衡。在利率問題上將根據更多數據作決定，原則上無需加息至太高水平，當局正在評估利率是否已對美國經濟構成有效限制。

他指出，聯儲局在壓低通脹方面，仍有漫長的路途要走，聯邦公開市場委員會（FOMC）的通脹前景展望並不支持減息。當局致力於實現2%的通脹目標，現在需要集中精力將通脹降下來。他指出，勞動力市場有一定的走弱，但仍然太強勁，現在需要集中精力將通脹降下來。

原文網址: FOMC｜聯儲局加息0.25厘　鮑威爾：通脹前景展望不支持減息 | 香港01 https://www.hk01.com/article/894315?utm_source=01articlecopy&utm_medium=referral
Summary:

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=50,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

The Federal Reserve raised interest rates by 0.25% as expected, and indicated that it might stop raising rates in the future. Jerome Powell, the chairman of the Federal Reserve, said that a recession was possible, but he believed it could be avoided. He also said that the Federal Reserve still had a long way to go in lowering inflation, and that if inflation remained high, the Federal Reserve


In [15]:
instruction = """
Please label the product based on the product name.
product name : "伊藤園綠茶530ml"

1. Please select the appropriate category from "咖啡類","飲料類","水類", or choose "Others" if the product does not fit into any of the listed categories.
2. Please provide some common labels that are applicable to the product, such as flavor, brand, key ingredient, and size (if applicable). 
Please avoid choosing duplicate labels and choose no more than fourlabels
3. Please return the results in JSON format with the following keys: "product_name", "category", and "common_labels".

"""

response = ai21.Completion.execute(sm_endpoint=endpoint_name,
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.2,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

{
"product_name": "伊藤園綠茶530ml",
"category": "飲料類",
"common_labels": [
"伊藤園",
"綠茶",
"530ml"
]
}


In [16]:
model.delete_model()